In [ ]:
# Azure Machine Learning SDK core
from azureml.core import Workspace
from azureml.core.model import Model

# Scikit-learn and others
from sklearn import datasets; iris = datasets.load_iris()
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\james\miniconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
  File "C:\Users\james\AppData\Local\Temp\ipykernel_1584\2441422215.py", line 2, in <module>
    from azureml.core import Workspace
ModuleNotFoundError: No module named 'azureml'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\james\miniconda3\envs\myenv\lib\site-packages\pygments\styles\__init__.py", line 89, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\james\miniconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
  File "c:\Users\james\miniconda3\envs\myenv\lib\site-packages\IPython\core\ultratb.py", line 1310, in structured_traceback
  File "c:\Users\ja